# TRAINING MODEL

In [4]:
import os
import warnings

# Import .py files
from dicom_utils import *
from helper import *

warnings.filterwarnings('ignore') # ignore tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # ignore tensorflow warnings

# Import PyTorch
import torch.nn as nn
import torch.optim as optim

# Import monai model
from monai.networks.nets import resnet10

In [5]:
# Set directory to root
os.chdir('/')
data_dir = "data/classification" 
save_dir = "group/bagel/Task_2/Final" 
csv_path = f"{data_dir}/train_labels.csv"

# Model variables
VAL_SIZE = 0.2

N_EPOCHS = 20

CONTRAST_KEYS = ['T1wCE']# ['FLAIR' , 'T1w', 'T1wCE', 'T2w']
BATCH_SIZE = [8] #8
NUM_IMAGES_3D = [21]

In [6]:
for contrast in CONTRAST_KEYS:
    # Define saving directory 
    save_dir_train = f"{save_dir}/models/{contrast}_model"
    # init dictionary and variable
    best_val_accuracy = 0
    all_results = {}
    for b_s in BATCH_SIZE:
        for n_img_3D in NUM_IMAGES_3D:
        
            # Clear CUDA memory
            torch.cuda.empty_cache()
        
            # Set the device to GPU
            device = torch.device("cuda")
            # Create a ResNet10 
            model = resnet10(spatial_dims=3, n_input_channels=1, num_classes=1)
            #model = resnet18(spatial_dims=3, n_input_channels=1, num_classes=1)
        
            # Create an Adam optimizer and a learning rate scheduler 
            optimizer = optim.Adam(model.parameters(), lr=0.0001)
            milestones = [5, 10, 15] if b_s == 4 else [10, 15]
            scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.5, last_epoch=-1, verbose=True)

            # Create a Binary Cross Entropy with Logits Loss as the criterion for binary classification
            criterion = nn.BCEWithLogitsLoss()
            
            # Create Data Loader
            set_random_seed(42)
            train_dl, validation_dl = get_train_data_generators(csv_path, contrast, n_img_3D, b_s, VAL_SIZE)
            #train_dl = get_train_data_generators(csv_path, contrast, n_img_3D, b_s, VAL_SIZE)
            
            # Training the model
            results, model_t, i_t, thresh_t, train_loss, train_accuracy, val_loss, val_accuracy = train_model(model,
                                                                                                          train_dl, 
                                                                                                          #train_dl, 
                                                                                                          validation_dl, 
                                                                                                          optimizer, scheduler, 
                                                                                                          criterion, N_EPOCHS, 
                                                                                                          device)
            if best_val_accuracy < val_accuracy:
                save_model(model_t, save_dir_train, i_t, scheduler, train_loss, train_accuracy, val_loss, val_accuracy, thresh_t,
                      n_img_3D, b_s, 'BCEWithLogitsLoss', contrast)
                best_val_accuracy = val_accuracy
                plot_metrics(results, save_dir_train)
            all_results[f"{contrast}_BS_{b_s}_NbIm_{n_img_3D}"] = results
            torch.cuda.empty_cache()
    plot_all(save_dir_train, all_results)

Adjusting learning rate of group 0 to 1.0000e-04.


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
